<a href="https://colab.research.google.com/github/erinmcmullen/CS167-Notes/blob/main/Day05_P1_Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS167: Day05 
## Part 1: Decision Trees

#### CS167: Machine Learning, J-Term 2023

Monday, January 9th, 2023 -- Session II (10:20 - 11:35)

[⏮ Day04 Part 2](https://github.com/merriekay/j23_cs167_notes/blob/main/Day04_P2_Metrics_Testing.ipynb) | [Day05 Part 1⏩]()

# Admin Stuff:

You should be working on: 
- Quiz #1, due tonight by 11:59pm
- [Notebook #3](https://classroom.github.com/a/960Y0T8W), released today, due tomorrow night 1/10 by 11:59pm.

## Helpful Links:
📆 [Course Schedule](https://docs.google.com/spreadsheets/d/e/2PACX-1vStj3FCEJqloUMLn2VtHa4yy1ILY6WvABhu4jd4cVUpPGkrx1mEjfTFmd77DMESR9HJ-8UBxgMDJL06/pubhtml?gid=0&single=true) | 🙋[PollEverywhere](https://pollev.com/meredithmoore011) | 📜 [Syllabus](https://analytics.drake.edu/~moore/j23_cs167/Syllabus.html)


## This Week Overview:

| **Date** | **Day of Week** | **Day of Class** | **Topic**                               | **Assigned** | **Due (11:59pm)** |
|----------|-----------------|------------------|-----------------------------------------|--------------|-------------------|
| 1/9/23   | M               | Day05            | Decision Trees Part I                   | [N3](https://classroom.github.com/a/960Y0T8W)           | Q1                |
|          |                 |                  | Decision Trees Part II                  |              |                   |
| 1/10/23  | T               | Day06            | Intro to Scikit Learn                   | N4           | N3                |
|          |                 |                  | Random Forests                          |              |                   |
| 1/11/23  | W               | Day07            | SVMs                                    | P1           | N4                |
|          |                 |                  | PCA                                     |              |                   |
| 1/12/23  | R               | Day08            | Intro to Neural Networks -- Percpetrons | N5           |                   |
|          |                 |                  | Perceptron Code                         |              |                   |
| 1/13/23  | F               | Day09            | Artifical Neural Networks               | Q2           | N5                |
|          |                 |                  | ANN Code                                |              |                   |


# Can't forget to add our data/libraries:
Make sure you download the `winequality-white.csv` file from Bb and put it in your Drive.

In [1]:
#run this cell if you're using Colab:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import the data:
#make sure the path on the line below corresponds to the path where you put your dataset.
import pandas as pd
import numpy as np
path = '/content/drive/MyDrive/Datasets/irisData.csv'
iris= pd.read_csv(path)
iris.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Quick Review:


__k-Nearest-Neighbor Algorihm__: Predict the _most commonly appearing_ class among the __k__ closest training examples.

## 3-Nearest-Neighbor Algorithm

> Wait... why did we skip 2-NN?

### What will a 3NN algorithm predict?

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day03_3NN_iris.png?raw=1" width=450/>
</div>

# Remember our kNN function?

In [3]:
def kNN(specimen, data, k):
    # write your code in here to make this function work
    # 1. calculate distances
    data['distance_to_new'] = np.sqrt(
    (specimen['petal length'] - data['petal length'])**2 
    +(specimen['sepal length'] - data['sepal length'])**2 
    +(specimen['petal width'] - data['petal width'])**2
    +(specimen['sepal width'] - data['sepal width'])**2)

    # 2. sort
    data.sort_values(['distance_to_new'], inplace=True)
    
    # 3. predict
    prediction = data.iloc[0:k]['species'].mode()

    return prediction.iloc[0]

In [4]:
new_iris = {}
new_iris['petal length'] = 5.1
new_iris['sepal length'] = 7.2
new_iris['petal width'] = 1.5
new_iris['sepal width'] = 2.5

kNN(new_iris, iris, 3)

'Iris-versicolor'

## kNN for Regression?

The only thing we need to change if our target variable is a real-valued number (continuous) is that rather than taking the `mode()` of the __k__ closest neighbors, we will take the `mean()` of the k closest neighbors.

## Are all neighbors created equal?

The way we've learned kNN so far, each neighbor gets an equal vote in the decision of what to predict.

Do we see any problems with this? If so, what?

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day04_wknn_motivation.png?raw=1" width = 500/>
</div>

# Weighted k-NNN Intuition:

In weighted kNN, the nearest k points are given a weight, and the weights are grouped by the target variable. The class with the largest sum of weights will be the class that is predicted. 

The intuition is to give more weight to the points that are nearby and less weight to the points that are farther away.
- distance-weighted voting

In w-kNN, we want to predict the target variable with the most weight, where the weight is defined by the inverse distance function.

## $w_{q,i} = \frac{1}{d(x_q, x_i)^2}$

> In English, you can read that as the __weight__ of a traning example is equal to 1 divided by the distance between the new instance and the traning example squared.

In [5]:
def w_knn(specimen, data, k):
    #calculate the distance
    data['distance_to_new'] = np.sqrt(
    (specimen['petal length'] - data['petal length'])**2 
    +(specimen['sepal length'] - data['sepal length'])**2 
    +(specimen['petal width'] - data['petal width'])**2
    +(specimen['sepal width'] - data['sepal width'])**2)

    
    # calculate the weights (remember, weights are 1/d^2)
    data['weight'] = 1/(data['distance_to_new']**2)
    
    # find the k closest neighbors
    data.sort_values(['distance_to_new'], inplace=True)
    neighbors = data.iloc[0:k]
    
    # use groupby to sum the weights of each species in the closest k
    results = neighbors.groupby(['species']).weight.sum()
    #print(results)
    #print(results.idxmax(), results.iloc[0])
    return results.idxmax()
    
    # return the class that has the largest sum of weight.

In [6]:
new_iris = {}
new_iris['petal length'] = 5.1
new_iris['sepal length'] = 7.2
new_iris['petal width'] = 1.5
new_iris['sepal width'] = 2.5

w_knn(new_iris, iris, 15)
#new_iris['petal length']

'Iris-versicolor'

## Z-score Normalization:


In [7]:
def z_score(columns, data):
    """
    takes in a list of columns to normalize using the z-score method
    Params:
        columns, a list of columns to normalize
        data, the dataframe, preferably a copy
    """
    for col in columns:
        #get the mean and std
        col_m = data[col].mean()
        col_s = data[col].std()
        
        data[col] = (data[col] - col_m)/col_s

In [8]:
# get the mean() and std() for each column of iris
pl_mean = iris['petal length'].mean()
pl_std = iris['petal length'].std()

pw_mean = iris['petal width'].mean()
pw_std = iris['petal width'].std()

sl_mean = iris['sepal length'].mean()
sl_std = iris['sepal length'].std()

sw_mean = iris['sepal width'].mean()
sw_std = iris['sepal width'].std()

#create a new dictionary with the normalized values
norm_iris = {}

norm_iris['petal length'] = (new_iris['petal length']-pl_mean)/pl_std
norm_iris['petal width'] = (new_iris['petal width']-pw_mean)/pw_std
norm_iris['sepal length'] = (new_iris['sepal length']-sl_mean)/sl_std
norm_iris['sepal width'] = (new_iris['sepal width']-sw_mean)/sw_std

## Cross-Validation

Don't train the model on the testing data!

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day04_cross_validation.png?raw=1" width=600/>
</div>

In [9]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/drive/MyDrive/Datasets/irisData.csv')

#shuffle the data "sampling" the full set in random order
shuffled_data = data.sample(frac=1, random_state=41)

#use the first 20 rows in the shuffled set as testing data 
#train with the rest.
test_data = shuffled_data.iloc[0:20]
train_data = shuffled_data.iloc[20:]
train_data.shape

(130, 5)

## Cross-Validation Metrics:

When doing cross-validation, how do we tell how well our model performed?

How can we measure it?

| **Type of ML** | **Metric**                | **Description**                                                                                              |
|----------------|---------------------------|:--------------------------------------------------------------------------------------------------------------|
| Classification | Accuracy                  | Number correct examples divided by total number                                                              |
| Classification | Confusion Matrix          | Detailed table of where our model got confused. |
| Regression     | Mean Absolute Error (MAE) | The average absolute distance from the target variable                                                       |
| Regression     | Mean Squared Error (MSE)  | The average squared distance from the target variable                                                        |
| Regression     | $R^2$                     | 1: perfectly fit data<br>0: same performance as guessing the mean of the target variable<br>-1: really bad.  |

## 💬 Review Q:

Imagine you are working with regression data, in which you need to compare two series of data (one predictions, the other answers) to determine how effective a value of k is in kNN:

Which metric do you think would work best and why?
- accuracy
- MSE
- MAE
- $R^2$

# ✨ New Material: 

# Generating $R^2$ Values:

In [10]:
import pandas as pd
actual_vals = pd.Series([.45,.76, .23,.98, .55])
predicted_vals = pd.Series([.55 ,.77, .28,.96, .57])

import sklearn.metrics #we'll talk more about sklearn tomorrow

r2_val = sklearn.metrics.r2_score(actual_vals, predicted_vals)
print("r^2 value: " , r2_val)

r^2 value:  0.9596044857108404


# [Notebook #3: Wine Quality 🍷](https://classroom.github.com/a/960Y0T8W)

Wines are typically rated (scored) on a range from 0-10.

Is there anything scientific or predictable to wine ratings?

Given measurable qualities of wine, can they predict the rating.

If I meaure, say 1600 white wines for alcohol, pH, citric acid, sulphates, etc, and then am given some random win, can we _predict_ it's rating?

In [11]:
import pandas as pd
import numpy as np
path = '/content/drive/MyDrive/Datasets/winequality-white.csv'
iris= pd.read_csv(path)
iris.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Exercise #1

1. Implement mean squared error (mse) algorithm and test it:

In [12]:
def mse(actual, predicted):
    """
    Input: 
        actual, a list of values
        predicted, a list of values
        (assume actual and predicted are the same len)
    Output: 
        the mean squared erro between the actual and predicted values
    """
    print("Your code goes here")

## Exercise #2

2. Implement cross validation--create training and testing data. Use approximately the first 320 rows in the shuffled set as testing data. 

In [13]:
wine_data = pd.read_csv('/content/drive/MyDrive/Datasets/winequality-white.csv')

#establish test_data and train_data here

## Exercise #3

3. Use the kNN implementation and the other code provided below to normalize your data. Test your model using several different values of k and __graphically show__ the results using your implmentation of __MSE__. 

## Exercise #4

4. Repeat the experiment and __graphically show the results__ using __$R^2$__. 

## Exercise #5

5. Describe your findings. What conclusions can you draw about the best number for k for the kNN algorithm for this dataset? __Explain why__. Use a markdown cell to explain your conclusions. 

# Decision Trees 🌳

### What is a tree in computer science?

__Tree__: a common data structure that simulates a hierarchical tree structure, with a root value and subtrees of children with a parent node, represented as a set of liked nodes. 

Where is a common place we interact with a tree datastructue on a daily basis?

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day05_decision_tree.png?raw=1" width=250/>
</div>

## What is a decision tree?
<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day05_dt_ex1.png?raw=1" width=600/>
</div>

## What is a decision tree?
<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day05_dt_ex2.png?raw=1" width=600/>
</div>

# Decision Tree Examples

We're going to switch to Google Slides here: [Decision Tree Example Slides](https://docs.google.com/presentation/d/1mQxycmE2gjC5i-Iwphpd_KAsaFxVDzJXkg_C2EBFGUQ/edit?usp=sharing)

## 🚨 Terminology Alert 🚨: `consistent` and `generalize`

Is this tree __consistent__ with the training examples?
- do all of the training examples get categorized appropriately?

Will this tree __generalize__ well to new examples?
- how well will new examples (test set) perform?

## Great, now how do I build (grow) a tree?

One algorithm that builds a decision tree is called the __ID3 Decision Tree Learning Algorithm__.

It goes like this: 
    
__Main ID3 Loop__:
1. Assign A to be the _best_ decision feature for the next node.
2. Assign A as decision feature for the node
3. For each possible attribute of A, create a new descendant of node
4. Soft training examples to leaf nodes
5. if Training examples are perfectly classified, the STOp, Else, iterate over new leaf nodes.

### What does 'best' mean? How can we tell which node is *best*?

## Selecting a feature: Polleverywhere

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day05_selecting_feature.png?raw=1" width=600/>
</div>

## Selecting a feature:
__idea__: a good feature splits the examples into __subsets that are as pure as possible__ (ideally) 'all positive' or 'all negative'

> patrons is a better choice--it gives more information about the classification.

## 🚨 Terminology Alert 🚨 `entropy`

__Entropy__: measure of impurity/randomness
- __high entropy__: more evenly split classes - highly unpredictable
- __low entropy__: mostly one class - highly predictable

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day05_entropy.png?raw=1" width=600/>
</div>

## Calculating Entropy Prior

__Prior Probability__:aka the 'prior'
- the split of the examples
- if I have 9 positive examples and 5 negative examples, my prior is:

$\langle 9/14, 5/14 \rangle \approx \langle 0.64, 0.36 \rangle$
> the prior probabilities must sum to 1

## Calculating Entropy:

Calculating the entropy when prior is $\langle P_1, ..., P_c\rangle$

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day05_entropy_calc.png?raw=1" width=600/>
</div>

- entropy of prior $\langle 0.5, 0.5 \rangle$ is $-0.5 \log_{2}(0.5) -0.5 \log_{2}(0.5)  = 1$

- entropy of prior $\langle 0.9, 0.1 \rangle$ is $-0.9 \log_{2}(0.9)-0.1 \log_{2}(0.1) \approx 0.47$

- entropy of prior $\langle 0.64, 0.36 \rangle$ is $-0.64 \log_{2}(0.64)-0.36 \log_{2}(0.36) \approx 0.94$

- entropy of prior $\langle 0.25, 0.25, 0.5 \rangle$ is $-0.25 \log_{2}(0.25)-0.25 \log_{2}(0.25)-0.5 \log_{2}(0.5) \approx 1.5 $

> The maximum entropy is $\log_{2}(k)$ where $k$ is the number of categories. It is not always bounded by 0 and 1

In [14]:
import math
math.log(3, 2) # log base 2 of 3

1.5849625007211563

## Code for Entropy:

In [15]:
import math
# here's the syntax for a log(Base 2)
for i in [.5,1,2,4,8,16,32,64]:
    print("log base 2 of", i, "is", math.log2(i))

log base 2 of 0.5 is -1.0
log base 2 of 1 is 0.0
log base 2 of 2 is 1.0
log base 2 of 4 is 2.0
log base 2 of 8 is 3.0
log base 2 of 16 is 4.0
log base 2 of 32 is 5.0
log base 2 of 64 is 6.0


In [16]:
import math
def entropy(percentage_list):
    #input: percentage_list consists of float values that sum to 1.0 
    #return: calculation of entropy for input percentages
    result = 0
    for percentage in percentage_list:
        if percentage != 0:
            result += -percentage*math.log2(percentage)
    return result

In [17]:
#example of a calculation of entropy
entropy([2/6,4/6])

0.9182958340544896

In [18]:
entropy([1/4, 1/4, 2/4])

1.5

In [19]:
import pandas
import numpy
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
path = '/content/drive/MyDrive/Datasets/restaurant.csv'  #'/content/drive/MyDrive/CS167/Datasets/restaurant.csv'
restaurant = pd.read_csv(path)

restaurant.head()

,alt,bar,fri,hun,pat,price,rain,res,type,est,target
0,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,Yes,No,Yes,Yes,Full,$,No,No,Thai,10-30,Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No


## Example: Step 1

Start by calculating the entropy of the example __before__ picking a feature:
- $\langle 0.5, 0.5 \rangle = 1$
<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day05_patrons_entropy.png?raw=1" width=600/>
</div>

## Example: Step 1


<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day05_patrons_entropy2.png?raw=1" width=600/>
</div>

In [21]:
#Patrons expected entropy
restaurant[['pat', 'target']].sort_values(['pat','target'])

,pat,target
1,Full,No
4,Full,No
8,Full,No
9,Full,No
3,Full,Yes
11,Full,Yes
6,None,No
10,None,No
0,Some,Yes
2,Some,Yes


In [22]:
#calculate entropy
entropy_patrons_full = entropy([4/6,2/6]) # 4/6 was No; 2/6 was Yes
entropy_patrons_none = entropy([2/2,0/2])
entropy_patrons_some = entropy([0/4,4/4])
print(entropy_patrons_full, entropy_patrons_none, entropy_patrons_some)

0.9182958340544896 0.0 0.0


## Expected Entropy

The __expected entropy__ for a feature is defined as the weighted sum of the entropies multiplied by the fraction of samples that belong to each set:

Example:

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day05_expected_entropy.png?raw=1" width=600/>
</div>

In [23]:
# calculate expected_entropy (weighted average)
expected_entropy_patrons = 6/12*entropy_patrons_full + 2/12*entropy_patrons_none + 4/12*entropy_patrons_some
expected_entropy_patrons

0.4591479170272448

## Information Gain:

The __information gain__ is *difference* between the entropy before the test and the expected entropy after the test. 

$Gain() = Entropy(before) - Expected Entropy (after)$

<div>
<img src="https://github.com/merriekay/j23_cs167_notes/blob/main/images/day05_IG_calc.png?raw=1" width=600/>
</div>

In [24]:
#calculate information gain (prior entropy - expected entropy)
information_gain_patrons = 1.0 - expected_entropy_patrons
information_gain_patrons

0.5408520829727552

# 💬 Group Exercise:

Calculate the __Information Gain__ for `hun` and `est`:

In [25]:
restaurant[['hun', 'target']].sort_values(['hun','target'])

,hun,target
4,No,No
6,No,No
8,No,No
10,No,No
2,No,Yes
1,Yes,No
9,Yes,No
0,Yes,Yes
3,Yes,Yes
5,Yes,Yes


In [26]:
EstNo = 4/5
EstYes = 1/5
EntropyNohun = entropy([EstNo, EstYes])
EsthunNo = 2/7
EsthunYes = 5/7
Entropyhun = entropy([EsthunNo, EsthunYes])

In [27]:
Expected_Entropy1 = (((5/12)*EntropyNohun) + ((7/12)*Entropyhun))
Final_Answer1 = 1 - Expected_Entropy1
Final_Answer1

0.19570962879973086

In [28]:
restaurant[['est', 'target']].sort_values(['est','target'])

,est,target
6,0-10,No
10,0-10,No
0,0-10,Yes
2,0-10,Yes
5,0-10,Yes
7,0-10,Yes
9,10-30,No
3,10-30,Yes
1,30-60,No
11,30-60,Yes


In [29]:
Est10No = 2/6
Est10Yes = 4/6
Entropy10 = entropy([Est10No, Est10Yes])
Est30No = 1/2
Est30Yes = 1/2
Entropy30 = entropy([Est30No, Est30Yes])
Est60No = 1/2
Est60Yes = 1/2
Entropy60 = entropy([Est60No, Est60Yes])
EstOver60No = 2/2
EstOver60Yes = 0/2
EntropyOver60 = entropy([EstOver60No, EstOver60Yes])

In [30]:
Expected_Entropy = (((6/12)*Entropy10) + ((2/12)*Entropy30) + ((2/12)*Entropy60) + ((2/12)*EntropyOver60))
Final_Answer = 1 - Expected_Entropy
Final_Answer

0.20751874963942196

In [31]:
#Estimated wait time will be a better node because it has a higher Entropy

In [32]:
#What is the information gain from B3
EstRepubY = 1/1
EstDemY = 0/1
EntropyY = entropy([EstRepubY, EstDemY])
EstRepubN = 0/3
EstDemN = 3/3
EntropyN = entropy([EstRepubN, EstDemN])
Expected_Entropy2 = (((1/5)*EntropyY) + ((4/5)*EntropyN))
starting_ent = entropy([3/5, 2/5])
Final_Answer2 = starting_ent - Expected_Entropy
Final_Answer2

0.17846934409409054